In [ ]:
# This is need to work with plotly in a notebook
import plotly.offline as py
py.init_notebook_mode(connected=False)
import plotly.graph_objs as go

import numpy as np
import pandas as pd
import os
import joblib

In [ ]:
root_path = '/Users/jessica/NaN/BayOptPy/tpot/Output/random_seed/100_generations/random_seed_020'
results_path = 'tpot_BANC_freesurf_vanilla_combi_100gen.dump'
results = joblib.load(os.path.join(root_path, results_path))

In [ ]:
def print_predecessor_generation(results, curr_generation):
    '''
    Print the predecessor's generation for all the models in a specific generation
    '''
    data_df = pd.DataFrame()
    for key in results['evaluated_individuals'][curr_generation].keys():
        predecessor = ''.join(results['evaluated_individuals'][curr_generation][key]['predecessor'])
        print(predecessor)
        for generation in range(curr_generation-1, -1, -1):
            if predecessor in results['evaluated_individuals'][generation].keys():
                mae = abs(results['evaluated_individuals'][generation][predecessor]['internal_cv_score'])
                print('Found predecessor in %d generation' %generation)
                print('')
                data_df = data_df.append({'model': key, 'predecessor': predecessor, 'generation': generation, 'mae': mae},
                                 ignore_index=True)

    # Check if there is any predecessor that is repeated in this generation
    print('List of repeated models')
    repeated_predecessors = set([x for x in list(data_df['predecessor']) if list(data_df['predecessor']).count(x)>1])
    print(repeated_predecessors)
    print('There are %d repeated predecessors' %len(repeated_predecessors))
    return data_df

In [ ]:
curr_gen = 50
data_df = print_predecessor_generation(results, curr_gen)

Reproduce the static plot in plotly

In [ ]:
trace1 = go.Scatter(
        y = data_df['mae'],
        mode='markers',
        text=data_df['generation'],
        hoverinfo = 'text',
        marker=dict(
            size=16,
            color = data_df['generation'], #set color equal to a variable
            colorscale='Viridis',
            showscale=True
    )
)
data_comp = [trace1]

# Configure title
layout_comp = go.Layout(
    title='MAE at 50 Generation',
    yaxis=dict(title='MAE'),
    xaxis=dict(title='Models')
)

fig_comp = go.Figure(data=data_comp, layout=layout_comp)
py.iplot(fig_comp, filename='scatter')